In [4]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu18

In [5]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-02 01:47:53--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.37MB/s    in 0.7s    

2020-11-02 01:47:54 (1.37 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [7]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.describe()

DataFrame[summary: string, marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [8]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
from pyspark.sql.functions import to_date
from pyspark.sql.types import *

In [27]:
votes_20_df = df.filter('total_votes > 19')
votes_20_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|
|         US|    1243530|R13C5INE1RTZP6|B00G66UDAO|     130471439|Sportspower For B...|            Baby|          2|    

In [11]:
percent_vote_df = votes_20_df.withColumn('percent_vote', votes_20_df['helpful_votes']/ votes_20_df['total_votes'])
top_50_votes_df = percent_vote_df.filter('percent_vote >= 0.50')
top_50_votes_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|      percent_vote|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|0.9120879120879121|
|         US|    1243530|R13C5INE1RTZP6|B00G

In [12]:
stars_5_df = top_50_votes_df.filter('star_rating == "5"')
stars_5_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|      percent_vote|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|0.9120879120879121|
|         US|   38665516|R33JJQWAUYBKD3|B005

In [13]:
vine_y_df = top_50_votes_df.filter('vine == "Y"')
vine_y_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|      percent_vote|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   21155796|R3N0XV9267NOXV|B00ZKTVI66|     155258416|Graco LiteRider C...|            Baby|          5|           41|         47|   Y|                N|No more disturbin...|This version of t...| 2015-08-31|0.8723404255319149|
|         US|   25087234| RSA6JQ346JZHZ|B00S

In [14]:
vine_y_5star_df = vine_y_df.filter('star_rating == "5"')

In [15]:
vine_n_df = top_50_votes_df.filter('vine == "N"')
vine_n_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|      percent_vote|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|0.9120879120879121|
|         US|    1243530|R13C5INE1RTZP6|B00G

In [16]:
vine_n_5star_df = vine_n_df.filter('star_rating == "5"')

In [17]:
# counts
votes = votes_20_df.count()

top_50_percent = top_50_votes_df.count()
total_5stars = stars_5_df.count()
vine_camp = vine_y_df.count()
vine_camp_5star = vine_y_5star_df.count()
not_vine_camp = vine_n_df.count()
vine_not_camp_5star = vine_n_5star_df.count()


In [18]:
star5_y_perc_all = (vine_camp_5star/top_50_percent) * 100
star5_y_perc_all

0.7903901083851783

In [19]:
star5_n_perc_all = (vine_not_camp_5star/ top_50_percent) * 100
star5_n_perc_all

47.082990961380446

In [20]:
vine_camp_total_5star = (vine_camp_5star/ total_5stars) * 100
vine_camp_total_5star

1.6510012259910094

In [21]:
not_vine_camp_total_5star = (vine_not_camp_5star/ total_5stars) * 100
not_vine_camp_total_5star

98.34899877400899

In [22]:
all_vine_camp_star5 = (vine_camp_5star/ vine_camp) * 100
all_vine_camp_star5

43.628509719222464

In [23]:
all_not_vine_camp_star5 = (vine_not_camp_5star/ not_vine_camp) * 100
all_not_vine_camp_star5

47.95170160197657

In [24]:
import pandas as pd

In [25]:
summary = {'Title' :['Total Number of Reviews', ' Number of 5 Star Votes', 'Percent of 5 Star Votes of Paid Campaign', 'Percent of 5 Star Votes of Not Part of Paid Campaign'], 
           'Value': [top_50_percent, total_5stars, all_vine_camp_star5, all_not_vine_camp_star5]}
summary_df = pd.DataFrame(summary)
summary_df

,Title,Value
0,Total Number of Reviews,25557.000000
1,Number of 5 Star Votes,12235.000000
2,Percent of 5 Star Votes of Paid Campaign,43.628510
3,Percent of 5 Star Votes of Not Part of Paid Ca...,47.951702
